In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import matplotlib.pyplot as plt
from torch.utils.data.dataset import Dataset
from scipy.stats import gaussian_kde
#torch.set_default_tensor_type('torch.FloatTensor')
#cuda0 = torch.device('cuda:0')

In [2]:
co_occ_counts=pd.read_csv("/mount/projekte/sfb-732/b9/users/prajit/co_occ.sm",delim_whitespace=True,header=None)
co_occ_counts

,0,1,2
0,sein_V,sein_V,1774610
1,sein_V,tot_A,22832
2,sein_V,Präsident_N,18369
3,sein_V,offiziell_A,16208
4,sein_V,Wirkung_N,20543
5,sein_V,Spruch_N,3277
6,sein_V,Licht_N,20082
7,sein_V,Ausgabe_N,11875
8,sein_V,geschehen_V,26381
9,sein_V,intim_A,1117


In [3]:
co_occ_counts

,0,1,2
0,sein_V,sein_V,1774610
1,sein_V,tot_A,22832
2,sein_V,Präsident_N,18369
3,sein_V,offiziell_A,16208
4,sein_V,Wirkung_N,20543
5,sein_V,Spruch_N,3277
6,sein_V,Licht_N,20082
7,sein_V,Ausgabe_N,11875
8,sein_V,geschehen_V,26381
9,sein_V,intim_A,1117


In [4]:
co_occ_counts.columns=['word','context','freq']
co_occ_counts=pd.pivot_table(co_occ_counts,index=['word'],columns='context',values='freq',aggfunc=np.sum)
co_occ_counts

context,#_N,$_N,%_N,***_N,---_N,--_A,--_N,--_V,-innen_N,-innen_V,...,überörtlich_A,üblich_A,übrig_A,übrigbleiben_V,übrige_N,übrigen_N,übriger_N,üppig_A,„_A,„_N
word,,,,,,,,,,,,,,,,,,,,,
#_N,234.0,2.0,8.0,NaN,NaN,5.0,402.0,NaN,NaN,NaN,...,NaN,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
$_N,2.0,1154.0,163.0,NaN,1.0,11.0,361.0,NaN,NaN,NaN,...,NaN,11.0,7.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN
%_N,8.0,163.0,116360.0,5.0,12.0,431.0,3133.0,10.0,20.0,10.0,...,10.0,425.0,891.0,13.0,47.0,9.0,2.0,17.0,NaN,NaN
***_N,NaN,NaN,5.0,184.0,NaN,5.0,110.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
---_N,NaN,1.0,12.0,NaN,20.0,4.0,659.0,5.0,NaN,NaN,...,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--_A,5.0,11.0,431.0,5.0,4.0,4128.0,15729.0,54.0,9.0,4.0,...,5.0,194.0,234.0,9.0,29.0,5.0,4.0,18.0,2.0,NaN
--_N,402.0,361.0,3133.0,110.0,659.0,15729.0,359384.0,1007.0,15.0,11.0,...,16.0,753.0,700.0,17.0,90.0,12.0,15.0,97.0,14.0,59.0
--_V,NaN,NaN,10.0,1.0,5.0,54.0,1007.0,38.0,NaN,NaN,...,NaN,7.0,2.0,NaN,NaN,NaN,1.0,1.0,NaN,NaN
-innen_N,NaN,NaN,20.0,NaN,NaN,9.0,15.0,NaN,230.0,140.0,...,NaN,1.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
tot_count=np.nansum(co_occ_counts)
co_occ_counts=co_occ_counts.applymap(lambda x: x/tot_count)
co_occ_counts["sum"]=co_occ_counts.sum(axis=1)
co_occ_counts[co_occ_counts.columns[:-1]]=co_occ_counts[co_occ_counts.columns[:-1]]/(co_occ_counts[co_occ_counts.columns[:-1]].sum(axis=0)**0.75)
co_occ_counts=co_occ_counts.drop('sum',axis=1).div(co_occ_counts["sum"], axis=0)
co_occ_counts=co_occ_counts.applymap(lambda x:np.log2(x))
co_occ_counts=co_occ_counts.fillna(0)
co_occ_counts[co_occ_counts<0]=0
co_occ_counts -= co_occ_counts.min()
co_occ_counts /= co_occ_counts.max()
co_occ_counts

context,#_N,$_N,%_N,***_N,---_N,--_A,--_N,--_V,-innen_N,-innen_V,...,überörtlich_A,üblich_A,übrig_A,übrigbleiben_V,übrige_N,übrigen_N,übriger_N,üppig_A,„_A,„_N
word,,,,,,,,,,,,,,,,,,,,,
#_N,1.000000,0.0,0.000000,0.000000,0.000000,0.0,0.309122,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
$_N,0.000000,1.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
%_N,0.000000,0.0,0.804983,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
***_N,0.000000,0.0,0.000000,1.000000,0.000000,0.0,0.144479,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
---_N,0.000000,0.0,0.000000,0.000000,0.927413,0.0,0.623354,0.540361,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
--_A,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
--_N,0.000000,0.0,0.000000,0.000000,0.091629,0.0,0.440924,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
--_V,0.000000,0.0,0.000000,0.000000,0.270356,0.0,0.362644,0.813331,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
-innen_N,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,1.000000,1.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000


In [6]:
co_occ_counts.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30150 entries, #_N to „_N
Columns: 30150 entries, #_N to „_N
dtypes: float64(30150)
memory usage: 6.8+ GB


In [7]:
CUDA = True
SEED = 1991
BATCH_SIZE = 64
LOG_INTERVAL = 10
EPOCHS = 10
torch.manual_seed(SEED)
if CUDA:
    torch.cuda.manual_seed(SEED)
ZDIMS = 300
msk = np.random.rand(len(co_occ_counts)) < 0.8
#train_df = co_occ_counts[msk]

#test_df = co_occ_counts[~msk]

In [8]:
class CoOccurenceDataset(Dataset):

    def __init__(self, df,transform=None):

        self.df = df.values
        self.names=df.index.tolist()
        self.df_len = len(self.names)
        self.transform = transform

    def __len__(self):
        return self.df_len

    def __getitem__(self, index):
        sample_name=self.names[index]
        
        
        #if self.transforms is not None:
        sample_tensor = torch.from_numpy(self.df[index]).float()

        if self.transform:
            sample_tensor = self.transform(sample_tensor)

        
        return sample_name,sample_tensor

In [10]:
#trainset = CoOccurenceDataset(train_df,transform=transforms.Compose([ToTensor()]))
#testset=CoOccurenceDataset(test_df)

dataset=CoOccurenceDataset(co_occ_counts)
#train_data = torch.utils.data.TensorDataset(trainset)
data_loader = torch.utils.data.DataLoader(dataset,shuffle=True,num_workers=1,batch_size=32,pin_memory=True)

#test_loader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=True,num_workers=1,pin_memory=True)

In [23]:
class AutoEncoder(nn.Module):
    def __init__(self, input_dim, encoding_dim):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Linear(input_dim, encoding_dim)
        self.decoder = nn.Linear(encoding_dim, input_dim)

    def forward(self, x):
        encoded = F.relu(self.encoder(x))
        decoded = self.decoder(encoded)
        return decoded, encoded

def l1_penalty(var):
    return torch.abs(var).sum()

def train(num_epochs, sparsity=True, l1_weight=1e-5):
    for epoch in range(1,  num_epochs + 1):
        train_loss=0
        for batch_idx,(_,data) in enumerate(data_loader):
        #print(data)
            data = Variable(data).cuda()
            optimizer.zero_grad()
        
        # enforcing sparsity with l1 reg
            if sparsity:
                decoder_out, encoder_out = model(data)
                mse_loss = F.mse_loss(decoder_out, data)
                l1_reg = l1_weight * l1_penalty(encoder_out)
                loss = mse_loss + l1_reg
            else:
                output, _ = model(data)
                loss = F.binary_cross_entropy_with_logits(output, data)
            # loss = F.mse_loss(output, data)
            
            loss.backward()
            train_loss += loss.item()
            optimizer.step()
            if batch_idx % 100 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(data_loader.dataset), 
                100. * batch_idx / len(data_loader),loss.data[0] / len(data)))

        print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(data_loader.dataset)))


In [28]:
input_dim = 30150
encoding_dim = 1

model = AutoEncoder(input_dim, encoding_dim)
model = model.cuda()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
num_epochs = 5


train(num_epochs)

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:40: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Train Epoch: 1 [0/30150 (0%)]	Loss: 0.010402
Train Epoch: 1 [3200/30150 (11%)]	Loss: 0.007663
Train Epoch: 1 [6400/30150 (21%)]	Loss: 0.005607
Train Epoch: 1 [9600/30150 (32%)]	Loss: 0.004063
Train Epoch: 1 [12800/30150 (42%)]	Loss: 0.002918
Train Epoch: 1 [16000/30150 (53%)]	Loss: 0.002068
Train Epoch: 1 [19200/30150 (64%)]	Loss: 0.001449
Train Epoch: 1 [22400/30150 (74%)]	Loss: 0.000998
Train Epoch: 1 [25600/30150 (85%)]	Loss: 0.000679
Train Epoch: 1 [28800/30150 (95%)]	Loss: 0.000459
====> Epoch: 1 Average loss: 0.0033
Train Epoch: 2 [0/30150 (0%)]	Loss: 0.000383
Train Epoch: 2 [3200/30150 (11%)]	Loss: 0.000255
Train Epoch: 2 [6400/30150 (21%)]	Loss: 0.000163
Train Epoch: 2 [9600/30150 (32%)]	Loss: 0.000113
Train Epoch: 2 [12800/30150 (42%)]	Loss: 0.000075
Train Epoch: 2 [16000/30150 (53%)]	Loss: 0.000051
Train Epoch: 2 [19200/30150 (64%)]	Loss: 0.000041
Train Epoch: 2 [22400/30150 (74%)]	Loss: 0.000030
Train Epoch: 2 [25600/30150 (85%)]	Loss: 0.000027
Train Epoch: 2 [28800/30150 (9

In [13]:
model.state_dict().keys()

odict_keys(['encoder.weight', 'encoder.bias', 'decoder.weight', 'decoder.bias'])

In [35]:
np.var(model.state_dict()['encoder.weight'][0].cpu().numpy())

2.0180023e-05

In [17]:
model.state_dict()['encoder.bias'].shape

torch.Size([300])

In [34]:
np.var(model.state_dict()['decoder.weight'][0].cpu().numpy())

0.0009978551

In [52]:
model.state_dict()['decoder.bias'].shape

torch.Size([30150])

In [36]:
np.var(co_occ_counts, axis=0).sum()

42828.224990252835

In [ ]:
count_series.describe()

In [ ]:
count_series.plot(kind="density",xlim=[0,30])

In [28]:
dataset.shape

AttributeError: 'CoOccurenceDataset' object has no attribute 'shape'

In [11]:
#torch.set_default_tensor_type('torch.cuda.DoubleTensor')
num_epochs = 50
batch_size = 32
learning_rate = 1e-3




def min_max_normalization(tensor, min_value, max_value):
    min_tensor = tensor.min()
    tensor = (tensor - min_tensor)
    max_tensor = tensor.max()
    tensor = tensor / max_tensor
    tensor = tensor * (max_value - min_value) + min_value
    return tensor



class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(30150, 3000),
            nn.ReLU(True),
            nn.Linear(3000, 300),
            nn.ReLU(True))
        self.decoder = nn.Sequential(
            nn.Linear(300, 3000),
            nn.ReLU(True),
            nn.Linear(3000,30150),
            nn.Sigmoid())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
    
    def self_encoder(self, x):
        x = self.encoder(x)
        return x


model = autoencoder().cuda()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate, weight_decay=1e-5)

for epoch in range(num_epochs):
    train_loss = 0
    for batch_idx,(_,data) in enumerate(data_loader):

        cur_head = Variable(data).cuda()
        # ===================forward=====================
        output = model(cur_head)
        loss = criterion(output, cur_head)
        MSE_loss = nn.MSELoss()(output, cur_head)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch+1,batch_idx * len(data),
                len(data_loader.dataset), 100. * batch_idx / len(data_loader),
                loss.item() / len(data)))

    #print('====> Epoch: {} Average loss: {:.4f}'.format(
        #epoch+1, train_loss / len(data_loader.dataset)))
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}, MSE_loss:{:.4f}'
          .format(epoch + 1, num_epochs, loss.data[0], MSE_loss.data[0]))

torch.save(model.state_dict(), './sim_autoencoder.pth')

Train Epoch: 1 [0/30150 (0%)]	Loss: 0.021662
Train Epoch: 1 [3200/30150 (11%)]	Loss: 0.001149
Train Epoch: 1 [6400/30150 (21%)]	Loss: 0.000563
Train Epoch: 1 [9600/30150 (32%)]	Loss: 0.000489
Train Epoch: 1 [12800/30150 (42%)]	Loss: 0.000491
Train Epoch: 1 [16000/30150 (53%)]	Loss: 0.000483
Train Epoch: 1 [19200/30150 (64%)]	Loss: 0.000459
Train Epoch: 1 [22400/30150 (74%)]	Loss: 0.000485
Train Epoch: 1 [25600/30150 (85%)]	Loss: 0.000561
Train Epoch: 1 [28800/30150 (95%)]	Loss: 0.000484
epoch [1/50], loss:0.0168, MSE_loss:0.0008


/usr/lib/python3.6/site-packages/ipykernel_launcher.py:71: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Train Epoch: 2 [0/30150 (0%)]	Loss: 0.000489
Train Epoch: 2 [3200/30150 (11%)]	Loss: 0.000479
Train Epoch: 2 [6400/30150 (21%)]	Loss: 0.000542
Train Epoch: 2 [9600/30150 (32%)]	Loss: 0.000500
Train Epoch: 2 [12800/30150 (42%)]	Loss: 0.000489
Train Epoch: 2 [16000/30150 (53%)]	Loss: 0.000454
Train Epoch: 2 [19200/30150 (64%)]	Loss: 0.000467
Train Epoch: 2 [22400/30150 (74%)]	Loss: 0.000466
Train Epoch: 2 [25600/30150 (85%)]	Loss: 0.000476
Train Epoch: 2 [28800/30150 (95%)]	Loss: 0.000548
epoch [2/50], loss:0.0183, MSE_loss:0.0009
Train Epoch: 3 [0/30150 (0%)]	Loss: 0.000465
Train Epoch: 3 [3200/30150 (11%)]	Loss: 0.000417
Train Epoch: 3 [6400/30150 (21%)]	Loss: 0.000500
Train Epoch: 3 [9600/30150 (32%)]	Loss: 0.000438
Train Epoch: 3 [12800/30150 (42%)]	Loss: 0.000414
Train Epoch: 3 [16000/30150 (53%)]	Loss: 0.000459
Train Epoch: 3 [19200/30150 (64%)]	Loss: 0.000449
Train Epoch: 3 [22400/30150 (74%)]	Loss: 0.000445
Train Epoch: 3 [25600/30150 (85%)]	Loss: 0.000469
Train Epoch: 3 [28800/3

Train Epoch: 17 [6400/30150 (21%)]	Loss: 0.000474
Train Epoch: 17 [9600/30150 (32%)]	Loss: 0.000443
Train Epoch: 17 [12800/30150 (42%)]	Loss: 0.000499
Train Epoch: 17 [16000/30150 (53%)]	Loss: 0.000435
Train Epoch: 17 [19200/30150 (64%)]	Loss: 0.000448
Train Epoch: 17 [22400/30150 (74%)]	Loss: 0.000435
Train Epoch: 17 [25600/30150 (85%)]	Loss: 0.000493
Train Epoch: 17 [28800/30150 (95%)]	Loss: 0.000407
epoch [17/50], loss:0.0114, MSE_loss:0.0005
Train Epoch: 18 [0/30150 (0%)]	Loss: 0.000504
Train Epoch: 18 [3200/30150 (11%)]	Loss: 0.000449
Train Epoch: 18 [6400/30150 (21%)]	Loss: 0.000385
Train Epoch: 18 [9600/30150 (32%)]	Loss: 0.000450
Train Epoch: 18 [12800/30150 (42%)]	Loss: 0.000438
Train Epoch: 18 [16000/30150 (53%)]	Loss: 0.000456
Train Epoch: 18 [19200/30150 (64%)]	Loss: 0.000378
Train Epoch: 18 [22400/30150 (74%)]	Loss: 0.000479
Train Epoch: 18 [25600/30150 (85%)]	Loss: 0.000437
Train Epoch: 18 [28800/30150 (95%)]	Loss: 0.000488
epoch [18/50], loss:0.0131, MSE_loss:0.0006
Trai

Train Epoch: 32 [9600/30150 (32%)]	Loss: 0.000373
Train Epoch: 32 [12800/30150 (42%)]	Loss: 0.000414
Train Epoch: 32 [16000/30150 (53%)]	Loss: 0.000419
Train Epoch: 32 [19200/30150 (64%)]	Loss: 0.000462
Train Epoch: 32 [22400/30150 (74%)]	Loss: 0.000445
Train Epoch: 32 [25600/30150 (85%)]	Loss: 0.000409
Train Epoch: 32 [28800/30150 (95%)]	Loss: 0.000421
epoch [32/50], loss:0.0155, MSE_loss:0.0008
Train Epoch: 33 [0/30150 (0%)]	Loss: 0.000427
Train Epoch: 33 [3200/30150 (11%)]	Loss: 0.000464
Train Epoch: 33 [6400/30150 (21%)]	Loss: 0.000402
Train Epoch: 33 [9600/30150 (32%)]	Loss: 0.000434
Train Epoch: 33 [12800/30150 (42%)]	Loss: 0.000478
Train Epoch: 33 [16000/30150 (53%)]	Loss: 0.000436
Train Epoch: 33 [19200/30150 (64%)]	Loss: 0.000425
Train Epoch: 33 [22400/30150 (74%)]	Loss: 0.000452
Train Epoch: 33 [25600/30150 (85%)]	Loss: 0.000391
Train Epoch: 33 [28800/30150 (95%)]	Loss: 0.000404
epoch [33/50], loss:0.0229, MSE_loss:0.0011
Train Epoch: 34 [0/30150 (0%)]	Loss: 0.000505
Train Ep

Train Epoch: 47 [12800/30150 (42%)]	Loss: 0.000479
Train Epoch: 47 [16000/30150 (53%)]	Loss: 0.000433
Train Epoch: 47 [19200/30150 (64%)]	Loss: 0.000467
Train Epoch: 47 [22400/30150 (74%)]	Loss: 0.000444
Train Epoch: 47 [25600/30150 (85%)]	Loss: 0.000441
Train Epoch: 47 [28800/30150 (95%)]	Loss: 0.000477
epoch [47/50], loss:0.0195, MSE_loss:0.0010
Train Epoch: 48 [0/30150 (0%)]	Loss: 0.000492
Train Epoch: 48 [3200/30150 (11%)]	Loss: 0.000445
Train Epoch: 48 [6400/30150 (21%)]	Loss: 0.000439
Train Epoch: 48 [9600/30150 (32%)]	Loss: 0.000428
Train Epoch: 48 [12800/30150 (42%)]	Loss: 0.000449
Train Epoch: 48 [16000/30150 (53%)]	Loss: 0.000457
Train Epoch: 48 [19200/30150 (64%)]	Loss: 0.000520
Train Epoch: 48 [22400/30150 (74%)]	Loss: 0.000468
Train Epoch: 48 [25600/30150 (85%)]	Loss: 0.000479
Train Epoch: 48 [28800/30150 (95%)]	Loss: 0.000505
epoch [48/50], loss:0.0144, MSE_loss:0.0006
Train Epoch: 49 [0/30150 (0%)]	Loss: 0.000444
Train Epoch: 49 [3200/30150 (11%)]	Loss: 0.000422
Train Ep

In [17]:
model.state_dict().keys()

odict_keys(['encoder.0.weight', 'encoder.0.bias', 'encoder.2.weight', 'encoder.2.bias', 'decoder.0.weight', 'decoder.0.bias', 'decoder.2.weight', 'decoder.2.bias'])

In [20]:
model.state_dict()['decoder.0.weight'].shape

torch.Size([2000, 300])